<a href="https://colab.research.google.com/github/bongho/Project_ASAC_5th_Review/blob/hoon/%ED%82%A4%EC%9B%8C%EB%93%9C_%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B6%9C_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 70.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

import torch
import pandas as pd
import numpy as np

# 데이터 준비 및 전처리

In [ ]:
review = pd.read_csv('/content/drive/MyDrive/ASAC 데이터 분석 과정/기업연계_프로젝트/encoded_review_df_filtered.csv')
review.head(1)

df = review

In [ ]:
### 가게 리뷰 groupby 통합

temp_df = df.drop(['review_id','user_id', 'date'], axis=1)
group_temp = temp_df.groupby('business_id').agg({
    'text': ' '.join,
    'stars': ['count','mean'],
    'useful': 'sum',
    'funny': 'sum',
    'cool': 'sum'
    }).reset_index()
group_temp.columns = ['business_id', 'text', 'count','stars','useful', 'funny', 'cool']

## 문장별 텍스트 리스트 컬럼 추가 버전 0805 이후

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ASAC 데이터 분석 과정/기업연계_프로젝트/review_encoding.csv')

In [ ]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,sentences
0,2348758,1575984,80739,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,"['If you decide to eat here, just be aware it ..."
1,3273772,1231666,98537,4.0,1,0,1,Cute interior and owner gave us tour of upco...,2017-01-14 20:54:15,['Cute interior and owner gave us tour of up...
2,5406063,898460,94651,4.0,1,0,1,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07,['I was really between 3 and 4 stars for this ...
3,44775,644900,92221,5.0,0,0,0,First time there and it was excellent!!! It fe...,2017-02-19 13:32:05,"['First time there and it was excellent!!!', ""..."
4,3891834,177785,92778,5.0,0,0,0,"Great burgers,fries and salad! Burgers have a...",2017-01-02 03:17:34,"['Great burgers,fries and salad!', 'Burgers ha..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2680161 entries, 0 to 2680160
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   review_id    int64  
 1   user_id      int64  
 2   business_id  int64  
 3   stars        float64
 4   useful       int64  
 5   funny        int64  
 6   cool         int64  
 7   text         object 
 8   date         object 
dtypes: float64(1), int64(6), object(2)
memory usage: 184.0+ MB


## Sampling 데이터로 돌리기

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ASAC 데이터 분석 과정/기업연계_프로젝트/final_dataset/sample_review.csv')

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30880 entries, 0 to 30879
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   review_id        30880 non-null  int64  
 1   user_id          30880 non-null  int64  
 2   business_id      30880 non-null  int64  
 3   stars            30880 non-null  float64
 4   useful           30880 non-null  int64  
 5   funny            30880 non-null  int64  
 6   cool             30880 non-null  int64  
 7   text             30880 non-null  object 
 8   date             30880 non-null  object 
 9   sentences        30880 non-null  object 
 10  sentence_counts  30880 non-null  int64  
dtypes: float64(1), int64(7), object(3)
memory usage: 2.6+ MB


# Siebert 활용하여 감성(sentiment) 분류

- 성능이 잘나오는 편
- 다양한 데이터셋에서 학습을 해서 분류를 하는 모델임으로 새로운 데이터셋에도 적용하면 잘 분류해줌

In [ ]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
b = sentiment_analysis('''Don't be stupid enough to eat here is. It is expensive and crappy.
Just eat at the cafeteria. My friends and myself came to humpty's to have a nice breakfast and it was horrid and a disaster.
the service was fine, the food was DESPICABLE.
My friend ordered the chicken louie the mashed potatoes tasted like acid reflux and the chicken was so rubbery she couldn't eat half of it. The ground beef in the Mexican scrambler was questionable too.
My other friend ordered the crunchy chicken parmesan the pasta was watery and definitely unappealing and the vegetables were cold and clearly frozen. we all feel sick.''')

print(b[0]['label'])
print(b[0]['score'])

In [ ]:
# 나중에 불러올 때 사용
import re
import json


def change_str(in_str):
    in_str = re.sub(r"'", '"',in_str )
    in_str = re.sub(r'"', '"',in_str)
    in_str = re.sub(r"True", 'true',in_str )
    in_str = re.sub(r"False", 'false',in_str )
    return in_str

In [ ]:
import ast

# 문자열로 된 리스트를 실제 리스트로 변환하는 함수
def parse_list_column(column):
    return column.apply(ast.literal_eval)

# 'sentences' 컬럼을 리스트로 변환
df['sentences'] = parse_list_column(df['sentences'])

In [ ]:
df['sentences'][0][1]

"I can't wait to check it out in person!"

### 전체 데이터에 적용

In [13]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import pickle
import os

# 중간 저장 파일 경로
checkpoint_path = "sentiment_analysis_checkpoint.pkl"

# 진행 상황 체크 및 복구
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'rb') as f:
        processed_results = pickle.load(f)
else:
    processed_results = {}

# 진행 상황 시각화 및 감정 분석 수행
for index, row in tqdm(df.iterrows(), total=len(df)):
    review_id = row['review_id']
    if review_id in processed_results:
        continue

    sentences = row['sentences']
    results = []
    for sentence in sentences:
        result = sentiment_analysis(sentence)[0]
        print(result)
        results.append((sentence, result['label'], result['score']))

    processed_results[review_id] = results

    # 중간 저장
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(processed_results, f)

# 결과를 데이터프레임에 반영
df['sentiment_analysis'] = df['review_id'].apply(lambda x: processed_results.get(x, []))

# # 결과 출력
# import ace_tools as tools; tools.display_dataframe_to_user(name="Sentiment Analysis Results", dataframe=df)
# print(df[['review_id', 'sentiment_analysis']])

{'label': 'POSITIVE', 'score': 0.9988837838172913}
{'label': 'POSITIVE', 'score': 0.9989039897918701}
{'label': 'POSITIVE', 'score': 0.9987855553627014}
{'label': 'POSITIVE', 'score': 0.9973286390304565}
{'label': 'POSITIVE', 'score': 0.9975830316543579}
{'label': 'POSITIVE', 'score': 0.9988253712654114}


 96%|█████████▌| 29573/30879 [3:43:30<23:03,  1.06s/it]

{'label': 'POSITIVE', 'score': 0.9972672462463379}
{'label': 'POSITIVE', 'score': 0.9985620379447937}
{'label': 'POSITIVE', 'score': 0.9986159801483154}


 96%|█████████▌| 29574/30879 [3:43:31<19:12,  1.13it/s]

{'label': 'POSITIVE', 'score': 0.9989162683486938}
{'label': 'POSITIVE', 'score': 0.9988197684288025}
{'label': 'NEGATIVE', 'score': 0.9989803433418274}


...

100%|█████████▉| 30877/30879 [4:09:02<00:02,  1.30s/it]

{'label': 'POSITIVE', 'score': 0.9987296462059021}
{'label': 'POSITIVE', 'score': 0.9987099170684814}
{'label': 'NEGATIVE', 'score': 0.9993033409118652}
{'label': 'NEGATIVE', 'score': 0.9890484809875488}
{'label': 'NEGATIVE', 'score': 0.9993312358856201}
{'label': 'POSITIVE', 'score': 0.9060736298561096}
{'label': 'NEGATIVE', 'score': 0.9994931221008301}
{'label': 'NEGATIVE', 'score': 0.9994888305664062}
{'label': 'NEGATIVE', 'score': 0.9995113611221313}
{'label': 'NEGATIVE', 'score': 0.9995087385177612}


100%|█████████▉| 30878/30879 [4:09:04<00:01,  1.38s/it]

{'label': 'NEGATIVE', 'score': 0.9994288086891174}
{'label': 'POSITIVE', 'score': 0.9986454844474792}
{'label': 'POSITIVE', 'score': 0.9989219903945923}
{'label': 'POSITIVE', 'score': 0.9986228942871094}
{'label': 'POSITIVE', 'score': 0.9988051652908325}


100%|██████████| 30879/30879 [4:09:05<00:00,  2.07it/s]


In [ ]:
df[df['review_id'] == 4531737]['sentences']

,sentences
23474,"[Brick and Mortar has been bookmarked for a long time, but due to the absurd amount of good restaurants in St Pete it took a while to get here...Big Mistake on our part as even in a great dining town like St Pete this one is a real standout...Located directly across Central from Cycle Brewing it takes up the bottom floor of an office building..the restaurant is not a big place, but when crowded they have a separate ""room"" they seat people in which is actually the lobby of the office building during business hours...I know the add on room sounds terrible, but that's where we sat and the extra space was actually quite nice...Brick and Mortar definitely has the hot concept in the restaurant business these days which is small plates with a drink menu specializing in Wine and Local Craft Beers...St Pete/Tampa Area has a lot of these type places, but Brick and Mortar does it better than most...went with a group of six who all like to order small plates and share so I was able to sample many dishes that included..Carpaccio with House Ravioli stuffed with a Poached Egg...Veal Meatballs over Creamy Parmesan Polenta...Slow Braised Octopus done 2 different ways...Curry and Spice Rubbed Smoked Pulled Pork Poutine..Carmelized Onion and Cheese Tart... and a Charred Romaine Heart....not a misfire was to be found in any of the six dishes with The Poutine Dish being my favorite...I like Poutine so much that The Canadians are close to making me a Honorary Citizen so I tend to order it the rare times I see it on menus...Brick and Mortars may be the best I ever had as it consisted of Truffle Fries covered with Red Curry Gravy..Smoked Pulled Pork..Queso Fresco...Pickled Onions..Cilantro..and Jalapeno...this dish just exploded with flavor...do yourself a favor if you go to Brick and Mortar and order this dish and you won't be sorry...besides the food they also had a fantastic wine and local craft beer selection and I was excited to be able to get a Cycle Cream and Sugar Please which is not available much outside the brewery..server was on point with good recommendations and was always there when we needed her...it was also a nice touch that the owner came by a few times to ask us how we were doing and how we enjoyed everything?..it's easy to see why everything is so good at Brick and Mortar and why it runs so smoothly when the owner is around and obviously cares so much..in a town full of great Restaurants Brick and Mortar is one of the best and I will definitely return sooner than later..this one rates an easy 5 stars on every Restaurant Measurable and deserves all it's success]"


In [14]:
df.to_csv('/content/drive/MyDrive/ASAC 데이터 분석 과정/기업연계_프로젝트/sentiment_analysis_result.csv', index=False)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,sentences,sentence_counts,sentiment_analysis
0,1443659,607521,108373,5.0,0,0,0,Found 51st Deli on Uber Eats and we loved it!!...,2017-01-14 21:23:10,[Found 51st Deli on Uber Eats and we loved it!...,2,[(Found 51st Deli on Uber Eats and we loved it...
1,1044648,182108,134961,5.0,0,0,0,Love love love this place for HH n dinner is g...,2017-08-07 23:48:51,[Love love love this place for HH n dinner is ...,11,[(Love love love this place for HH n dinner is...
2,5693786,749220,54049,5.0,0,0,0,Not sure the significance behind the name and ...,2018-09-25 14:49:55,[Not sure the significance behind the name and...,4,[(Not sure the significance behind the name an...
3,1166589,572170,9263,2.0,1,0,0,The service is going to be slooooww. And the f...,2017-10-02 01:52:47,"[The service is going to be slooooww., And the...",7,"[(The service is going to be slooooww., NEGATI..."
4,6678286,816928,108373,5.0,0,0,0,Much more than a deli! Wife and I had the Phil...,2017-12-29 20:18:59,"[Much more than a deli!, Wife and I had the Ph...",6,"[(Much more than a deli!, POSITIVE, 0.99826186..."
...,...,...,...,...,...,...,...,...,...,...,...,...
30875,5143901,1772582,22595,1.0,0,0,0,"Brand new but filthy. Trash overflowing, dirty...",2021-11-07 23:54:35,"[Brand new but filthy., Trash overflowing, dir...",5,"[(Brand new but filthy., NEGATIVE, 0.998448133..."
30876,2367318,995712,133866,3.0,8,2,3,Stopped in for dinner and drinks on a Saturday...,2018-03-25 15:30:29,[Stopped in for dinner and drinks on a Saturda...,15,[(Stopped in for dinner and drinks on a Saturd...
30877,6085901,1283829,84658,5.0,1,0,2,"Came here for lunch with my girlfriend, place ...",2018-08-10 04:52:50,"[Came here for lunch with my girlfriend, place...",4,"[(Came here for lunch with my girlfriend, plac..."
30878,5916423,1410939,123182,1.0,0,0,0,Used to go here every week. Won't go again. Ma...,2020-06-03 00:49:05,"[Used to go here every week., Won't go again.,...",10,"[(Used to go here every week., POSITIVE, 0.998..."


In [19]:
processed_results

{1443659: [('Found 51st Deli on Uber Eats and we loved it!!!',
   'POSITIVE',
   0.9988909363746643),
  ("I can't wait to check it out in person!", 'POSITIVE', 0.9988786578178406)],
 1044648: [('Love love love this place for HH n dinner is good too!',
   'POSITIVE',
   0.9989288449287415),
  ('Its ok for brunch, not as great as what people praise it to be.',
   'NEGATIVE',
   0.9975383281707764),
  ('The Asian Nachos are the dish to get for HH.',
   'POSITIVE',
   0.998580813407898),
  ('Its an entree itself!', 'POSITIVE', 0.9962144494056702),
  ('The other HH dishes are great too.', 'POSITIVE', 0.9986339211463928),
  ('The creme brulee is amazing as with their other desserts.',
   'POSITIVE',
   0.9988223910331726),
  ('The blueberry mojito is awesome along with the mango sling.',
   'POSITIVE',
   0.9988654851913452),
  ('Great place to celebrate birthdays and such.',
   'POSITIVE',
   0.9988657236099243),
  ('Nice feel to it.', 'POSITIVE', 0.998691976070404),
  ('Bar area gets crowd

### 병렬처리..

In [ ]:

# 감정 분석 파이프라인 설정 (GPU 사용)
device = 0 if torch.cuda.is_available() else -1
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english", device=device)

# 중간 저장 파일 경로
checkpoint_path = "sentiment_analysis_checkpoint.pkl"

# 진행 상황 체크 및 복구
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'rb') as f:
        processed_results = pickle.load(f)
else:
    processed_results = {}

# 감정 분석 함수
def analyze_sentiment_batch(sentences):
    return sentiment_analysis(sentences)

# 데이터프레임의 각 리뷰에 대해 감정 분석 수행
rows_to_process = [(index, row) for index, row in df.iterrows() if row['review_id'] not in processed_results]

for index, row in tqdm(rows_to_process, total=len(rows_to_process), desc="Processing Reviews"):
    review_id = row['review_id']
    sentences = row['sentences']
    results = analyze_sentiment_batch(sentences)
    processed_results[review_id] = [(sentence, result['label'], result['score']) for sentence, result in zip(sentences, results)]

    # 중간 저장
    with open(checkpoint_path, 'wb')  as f:
        pickle.dump(processed_results, f)

# 결과를 데이터프레임에 반영
df['sentiment_analysis'] = df['review_id'].apply(lambda x: processed_results.get(x, []))

# 결과 출력
import ace_tools as tools; tools.display_dataframe_to_user(name="Sentiment Analysis Results", dataframe=df)
print(df[['review_id', 'sentiment_analysis']])

NameError: name 'os' is not defined

In [ ]:
df[df['review_id'] == 4344482]['sentences']

Series([], Name: sentences, dtype: object)

In [ ]:
df = df.drop(df[df['review_id'] == 4344482].index)

In [ ]:
df['']

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,sentences,sentence_counts
0,1443659,607521,108373,5.0,0,0,0,Found 51st Deli on Uber Eats and we loved it!!...,2017-01-14 21:23:10,[Found 51st Deli on Uber Eats and we loved it!...,2
1,1044648,182108,134961,5.0,0,0,0,Love love love this place for HH n dinner is g...,2017-08-07 23:48:51,[Love love love this place for HH n dinner is ...,11
2,5693786,749220,54049,5.0,0,0,0,Not sure the significance behind the name and ...,2018-09-25 14:49:55,[Not sure the significance behind the name and...,4
3,1166589,572170,9263,2.0,1,0,0,The service is going to be slooooww. And the f...,2017-10-02 01:52:47,"[The service is going to be slooooww., And the...",7
4,6678286,816928,108373,5.0,0,0,0,Much more than a deli! Wife and I had the Phil...,2017-12-29 20:18:59,"[Much more than a deli!, Wife and I had the Ph...",6
...,...,...,...,...,...,...,...,...,...,...,...
30875,5143901,1772582,22595,1.0,0,0,0,"Brand new but filthy. Trash overflowing, dirty...",2021-11-07 23:54:35,"[Brand new but filthy., Trash overflowing, dir...",5
30876,2367318,995712,133866,3.0,8,2,3,Stopped in for dinner and drinks on a Saturday...,2018-03-25 15:30:29,[Stopped in for dinner and drinks on a Saturda...,15
30877,6085901,1283829,84658,5.0,1,0,2,"Came here for lunch with my girlfriend, place ...",2018-08-10 04:52:50,"[Came here for lunch with my girlfriend, place...",4
30878,5916423,1410939,123182,1.0,0,0,0,Used to go here every week. Won't go again. Ma...,2020-06-03 00:49:05,"[Used to go here every week., Won't go again.,...",10


In [ ]:
df[df['review_id'] == 4344482]['sentences']

1434    [午餐和晚餐菜单不一样，午餐简餐，晚上是正餐豪华一些，去的时候是中午后厨还是给做了几道主菜。...
Name: sentences, dtype: object

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")

# 512 토큰을 넘는 문장 체크 함수
def check_token_length(sentences):
    for sentence in sentences:
        tokens = tokenizer.encode(sentence, add_special_tokens=True)
        if len(tokens) > 512:
            print(f"Sentence exceeds 512 tokens: {sentence}")
            return True
    return False

# 각 리뷰의 문장을 체크
for index, row in df.iterrows():
    if check_token_length(row['sentences']):
        print(f"Review ID {row['review_id']} contains a sentence that exceeds 512 tokens.")

Token indices sequence length is longer than the specified maximum sequence length for this model (625 > 512). Running this sequence through the model will result in indexing errors


Sentence exceeds 512 tokens: 午餐和晚餐菜单不一样，午餐简餐，晚上是正餐豪华一些，去的时候是中午后厨还是给做了几道主菜。  【服务】热情 路过的人都会询问你的需求 【环境】靠外部有吧台，小卡座，往里走有正经一些 ，装修还可以，不十分豪华但也得体 【菜品】 *烤生蚝：进口生蚝，具体哪里的还真不知道，无异味，上面有酥酥的菜，不错的前菜，一盘三个 *带子：三个巨大的带子，轻微煎过，很嫩，还饱有汁水，但味道我觉得单一了一些，下面有超过的什锦蔬菜丁 *海鲜墨鱼意面：奶味+虾味，清淡倒也回味，墨鱼面爽滑。配料十分丰富，大虾，龙虾钳肉，两颗带子，意面上的带子有一点点不新鲜不如另一个主菜  还会再来，记得想吃丰富的记得一定要晚上去～
Review ID 4344482 contains a sentence that exceeds 512 tokens.


# 리뷰 요약 (T5)

- Text to Text Transfer Transformer
- 통일된 접근법: T5는 텍스트를 텍스트로 변환하는 일관된 접근 방식을 사용하여 키워드 생성 등 다양한 작업을 수행
- 다양한 작업 수행 능력: 텍스트 요약, 번역, 질문 생성, 키워드 생성 등 여러 작업을 하나의 모델로 수행

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from multiprocessing import Pool, set_start_method, cpu_count
from tqdm import tqdm
import numpy as np
import torch

# 'spawn' 시작 방법 설정
set_start_method('spawn', force=True)

# 1. 토크나이저와 모델 로드
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# 2. 텍스트 요약 함수 정의
def summarize_text(text):
    input_text = "summarize: " + text
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    outputs = model.generate(input_ids=input_ids, max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

## 결과 확인

In [ ]:
summarize_text(df['text'][0])

'we have tried it multiple times, because I want to like it! the food is good, but it takes a very long time to come out.'

In [ ]:
group_temp.columns = ['business_id', 'text', 'count','stars','useful', 'funny', 'cool']

## 가게 리뷰 합산 요약

In [ ]:
group_temp['text'][2]
## 모델의 metric
# ROGUE
# 원문과 요약의 공통 비율
# ROGUE 1 : 1-gram안에서의 공통 비율

'Server was ignorant, and did the bare minimum. My boyfriend and I had the Swiss chicken wraps, and the lettuce was soggy and old, and the tomatoes were extremely underripe. I brought the issue to the waitresses attention and nothing was done to fix the issue. Overall a not so great experience. The fries & gravy were tolerable, but not enough for us to return. I definitely do not recommend this establishment. My wife and I had some time to spare before going to the Royal Alex hospital so we decided on Humpty\'s just across the street at 9AM. My wife ordered the waffle and I had the Canadian skillet. Coffee was plentiful. Orders arrived after 12 minutes which was ok considering the place was only about 1/4 full. Waffle topped with overly sweet whipped cream and some strawberries. I had a full skillet but 3/4 were potatoes. Couldn\'t really tell how many eggs were in the skillet as there was a very unusual and off tasting yellow sauce. Needed a magnifying glass to find the bacon. I left 

In [ ]:
df['text']

0          If you decide to eat here, just be aware it is...
1          Cute interior and owner (?) gave us tour of up...
2          I was really between 3 and 4 stars for this on...
3          First time there and it was excellent!!! It fe...
4          Great burgers,fries and salad!  Burgers have a...
                                 ...                        
2680156    The supper club is ridiculously expensive. So,...
2680157    Took a colleague here for dinner as we were tr...
2680158    This place never fails the food is absolutely ...
2680159    This is a good pizza option - they deliver thr...
2680160    Located in the 'Walking District' in Nashville...
Name: text, Length: 2680161, dtype: object

# 리뷰 키워드 추출 (T5)

### 과학 아티클 fine-tuning 모델

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("Voicelab/vlt5-base-keywords")
tokenizer = T5Tokenizer.from_pretrained("Voicelab/vlt5-base-keywords")

task_prefix = "Keywords: "
# inputs = [
#     "Christina Katrakis, who spoke to the BBC from Vorokhta in western Ukraine, relays the account of one family, who say Russian soldiers shot at their vehicles while they were leaving their village near Chernobyl in northern Ukraine. She says the cars had white flags and signs saying they were carrying children.",
#     "Decays the learning rate of each parameter group by gamma every step_size epochs. Notice that such decay can happen simultaneously with other changes to the learning rate from outside this scheduler. When last_epoch=-1, sets initial lr as lr.",
#     "Hello, I'd like to order a pizza with salami topping.",
# ]

for sample in df.loc[:10,'text']:
    input_sequences = [task_prefix + sample]
    input_ids = tokenizer(
        input_sequences, return_tensors="pt", truncation=True
    ).input_ids
    output = model.generate(input_ids, no_repeat_ngram_size=3, num_beams=4)
    predicted = tokenizer.decode(output[0], skip_special_tokens=True)
    results.append({"text": sample, "keywords": predicted})


results_df = pd.DataFrame(results)
#    print(sample, "\n --->", predicted)

#First time there and it was excellent!!!
#It feels like your are entering someone's home.
#The waiters there funny and nice. The food come out very quickly and it is phenomenal!!!
#Definitely will be going back to this place.

# ---> food, vegetable, waiters


config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. 

The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker. 
 ---> NJ, food, quicker, waitstaff
Cute interior and owner (?) gave us tour of upcoming patio/rooftop area which will be great on beautiful days like today. Cheese curds were very good and very filling. Really like that sandwiches come w salad, esp after eating too many curds! Had the onion, gruyere, tomato sandwich. Wasn't too much cheese which I liked. Needed something else...pepper jelly maybe. Would like to see more menu options added such as salads w fun cheeses. Lots of beer

전체 합치기

In [ ]:
for sample in group_temp.loc[:10,'text']:
    input_sequences = [task_prefix + sample]
    input_ids = tokenizer(
        input_sequences, return_tensors="pt", truncation=True
    ).input_ids
    output = model.generate(input_ids, no_repeat_ngram_size=3, num_beams=4)
    predicted = tokenizer.decode(output[0], skip_special_tokens=True)
    print(sample, "\n --->", predicted, "\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Ate here for the 1st time on Saturday 08/07/2020. 
The food was ok, had the Lobster Boil . The Lobster was ok for a soft shelled Lobster. The Crabs were soggy and mushy. The Clams were gummy and snotty . The headed shrimp were large and tasty. All the items were precooked, obviously. The fish spread was decent , also had the Fried Shrimp and Shrimp scampi  which were decent. BYOB ... ... The atmosphere was nice except for the zillion Flies. I'll be back with a six pack Absolutely fantastic!!!!  I'll be back for sure. Fried lobster tail was delicious. Five star for sure. I was sadly disappointed in my experience. We ordered crabs and they were extremely small, watery and over half were bad(sick crabs), we ordered beans and rice which lacked flavor and literally had one bean in it that you can see from the picture. The coleslaw also lacked taste. We ask for grouper nuggets and a few were not cooked all the way. So given the food experience it was nothing like the reviews we read. Why I a

### ✅ 레스토랑 기반 fine-tuning 모델
- NUSTM/restaurant-t5-base

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("NUSTM/restaurant-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("NUSTM/restaurant-t5-base")

input_ids = tokenizer(
   "The pizza here is delicious!!", return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids=input_ids,num_beams=4)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# T5 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("NUSTM/restaurant-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("NUSTM/restaurant-t5-base")

# 입력 텍스트
input_text = '''Cute interior and owner gave us tour of upcoming patio/rooftop area which will be great on beautiful days like today.
Cheese curds were very good and very filling.
Really like that sandwiches come w salad, esp after eating too many curds!
Had the onion, gruyere, tomato sandwich. Wasn't too much cheese which I liked.
Needed something else...pepper jelly maybe. Would like to see more menu options added such as salads w fun cheeses.
Lots of beer and wine as well as limited cocktails.
Next time I will try one of the draft wines'''

input_ids = tokenizer(input_text, return_tensors="pt").input_ids
# 텍스트 생성 (키워드 추출)
outputs = model.generate(input_ids, num_beams=4)
predicted_keywords = tokenizer.decode(outputs[0], skip_special_tokens=True,)

print(f"Predicted Keywords: {predicted_keywords}")

In [ ]:
input_text = '''If you decide to eat here, \
just be aware it is going to take about 2 hours from beginning to end.
We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience.
The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant.
We have just had too many experiences where we spent way too long waiting.
We usually opt for another diner or restaurant on the weekends, in order to be done quicker'''

input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# 텍스트 생성 (키워드 추출)
outputs = model.generate(input_ids, num_beams=4)
predicted_keywords = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Predicted Keywords: {predicted_keywords}")

In [ ]:
input_text = "Don't be stupid enough to eat here is. It is expensive and crappy.   Just eat at the cafeteria. My friends and myself came to humpty's to have a nice breakfast and it was horrid and a disaster. the service was fine, the food was DESPICABLE. My friend ordered the chicken louie the mashed potatoes tasted like acid reflux and the chicken was so rubbery she couldn't eat half of it. The ground beef in the Mexican scrambler was questionable too. My other friend ordered the crunchy chicken parmesan the pasta was watery and definitely unappealing and the vegetables were cold and clearly frozen. we all feel sick. "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# 텍스트 생성 (키워드 추출)
outputs = model.generate(input_ids)
predicted_keywords = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Predicted Keywords: {predicted_keywords}")

Predicted Keywords: humpty's humpty


# 리뷰 키워드 추출(KeyBert)

- 핵심 문장을 먼저 뽑고 여기에서 키워드 추출
- 양방향 문맥 이해: BERT는 양방향 문맥을 고려하여 텍스트를 이해하는 것에 효과적이다.
- 주로 텍스트 분류, 질문 답변, 문장 관계 예측에 사용됨.


[참고] https://iopscience.iop.org/article/10.1088/1742-6596/1992/4/042077/pdf

In [ ]:
!pip install keybert

# Bertopic

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Bert 모델과 토크나이저 로드
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def extract_key_sentences(text, num_sentences=3):
    sentences = text.split('.')
    embeddings = []

    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors='pt')
        outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy())

    similarities = cosine_similarity(embeddings)
    key_sentences = sorted(range(len(similarities)), key=lambda i: similarities[i].sum(), reverse=True)[:num_sentences]

    return [sentences[i] for i in key_sentences]

In [ ]:
extract_key_sentences(group_temp['text'][0])

[' \n\nI ordered to go my order was ready in 10 minutes \n\nThe staff was very polite \n\nMy food was hot, fresh and tasty! The potion of coleslaw was huge an my lobster was a nice size',
 " Can't wait to return!! Great little spot and best of all BYOB !!!!\nLove it \n\nIs the food was fresh coat and they brought in the stone crabs that night on ice !  The boiled shrimp are great and the outside seating makes it feel very old Florida which is cool  , oysters Rockefeller was  pretty good",
 " The establishment makes you feel so comfortable and warm you don't want to leave! I had the Cuban Empanadas's and they were delicious  I ordered the baked beans also and they were some of the best I've ever had"]

In [ ]:
extract_key_sentences(df['text'][0])

[' \n\nThe food is good, but it takes a very long time to come out',
 ' We usually opt for another diner or restaurant on the weekends, in order to be done quicker',
 'If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end']

In [ ]:
df['text'][0]

"If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker."

### LDA 사용하여 토픽 추출

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 리뷰 데이터를 LDA를 사용하여 토픽 모델링
def topic_modeling(reviews, num_topics=5, num_keywords=5):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(reviews)

    lda = LatentDirichletAllocation(n_components=num_topics)
    lda.fit(X)

    topics = []
    for idx, topic in enumerate(lda.components_):
        keywords = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-num_keywords - 1:-1]]
        topics.append(keywords)

    return topics

#### 불용어 처리 전 예시

In [ ]:
# # 예시 데이터
reviews = [
    "The food was great and the service was excellent.",
    "I love the ambiance but the food was just okay.",
    "The pasta was delicious and the wine selection was superb."
]

# 토픽별 키워드 추출
topics = topic_modeling(reviews, num_topics=3, num_keywords=5)

for idx, topic in enumerate(topics):
    print(f"Topic {idx+1}: {topic}")


extract_keywords(reviews[0])

Topic 1: ['the', 'was', 'wine', 'superb', 'selection']
Topic 2: ['food', 'and', 'the', 'was', 'ambiance']
Topic 3: ['was', 'the', 'food', 'and', 'great']


['food', 'great', 'was', 'and', 'the', 'service']

#### 불용어 처리 후

In [ ]:
# 예시 데이터
reviews = [
    "The food was great and the service was excellent.",
    "I love the ambiance but the food was just okay.",
    "The pasta was delicious and the wine selection was superb."
]

preprocessed_reviews = [preprocess_text(review) for review in reviews]
topics_v2 = topic_modeling(preprocessed_reviews, num_topics=3, num_keywords=5)
for idx, topic in enumerate(topics_v2):
    print(f"Topic {idx+1}: {topic}")

extract_keywords(preprocessed_reviews[0])

Topic 1: ['food', 'love', 'okay', 'ambiance', 'excellent']
Topic 2: ['wine', 'superb', 'selection', 'pasta', 'delicious']
Topic 3: ['service', 'great', 'excellent', 'love', 'ambiance']


['food', 'great', 'excellent', 'service']

#### 실제 리뷰 데이터로 실험

In [ ]:
preprocessed_reviews = [preprocess_text(review) for review in df.loc[:1,'text']]

topics_v2 = topic_modeling(preprocessed_reviews, num_topics=3, num_keywords=5)
for idx, topic in enumerate(topics_v2):
  print(f"Topic {idx+1}: {topic}")

Topic 1: ['like', 'curds', 'cheese', 'come', 'good']
Topic 2: ['waitstaff', 'want', 'decide', 'way', 'done']
Topic 3: ['waitstaff', 'want', 'decide', 'way', 'done']
